In [34]:
import matplotlib.pyplot as plt
import nturl2path
import numpy as np
from brian2 import *

start_scope()
#assume only exist acetylcholine acceptor (excitor)

# parameters
taum   = 15*ms   # membrane time constant
Cm     = 0.1*ms #membrane capacitance
g_L    = Cm/taum   # membrane leak conductance
E_l    = -0.07 # membrane leak reversal potential
E_ach    = 0  # acetylcholine reversal potential
tau_ach  = 20*ms # acetylcholine excitatory synaptic time constant

E_GABAA  = -0.07 # GABAA reversal potential
tau_GABAA = 5*ms # GABAA synaptic time constant

Vr     = E_l     # reset potential
Vth    = -0.05   # spike threshold
Vs     = 20   # spiking potential
sigma = 0.001



w_e    = 1  	 # excitatory synaptic weight (units of g_L)
w_EPG = 0.11 #weight between EPG
w_EI = 0.47 #weight EPG to Ihn
w_IE = 12.5 #weight Inh to EPG
w_RR = 0.005 #weight R to R
w_EP = 0.01
w_PE = 0.01
# s_ace gating variable

#neuron subgroup names
subname_EPG = ['EPG0','EPG1','EPG2','EPG3','EPG4','EPG5','EPG6','EPG7','EPG8','EPG9','EPG10','EPG10','EPG11','EPG12','EPG13','EPG14','EPG15','EPG16','EPG17']
#number of neurons in each group
subnum_EPG = [3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
subname_PEN = ['PEN0','PEN1','PEN2','PEN3','PEN4','PEN5','PEN6','PEN7','PEN8','PEN9','PEN10','PEN11','PEN12','PEN13','PEN14','PEN15']
subnum_PEN = [3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
nn_cum = [0]
nn_cum.extend(np.cumsum(subnum_EPG)) #cumulative numbers for subgroup indexes
np_cum = [0]
np_cum.extend(np.cumsum(subnum_PEN))

# model equations
eqs = '''
dv/dt = ( Isyn + I + g_L*(E_l - v)) / Cm +sigma*sqrt(2/taum)*xi: 1 (unless refractory)
I : 1
Isyn = Isyn_EE + Isyn_EI + Isyn_IE + Isyn_RR + Isyn_EP + Isyn_PE : 1
Isyn_EE : 1
Isyn_EI:1
Isyn_IE:1
Isyn_RR:1
Isyn_EP:1
Isyn_PE:1
'''
Ach_eqs_EE = '''
ds_ach/dt = -s_ach/tau_ach : 1 (clock-driven)
Isyn_EE_post = -s_ach*(v-E_ach):1 (summed)
wach : 1 
'''
Ach_eqs_EI = '''
ds_ach/dt = -s_ach/tau_ach : 1 (clock-driven)
Isyn_EI_post = -s_ach*(v-E_ach):1 (summed)
wach : 1 
'''
GABA_eqs_IE = '''
ds_GABAA/dt = -s_GABAA/tau_GABAA : 1 (clock-driven)
Isyn_IE_post = -s_GABAA*(v-E_GABAA):1 (summed)
wach : 1
'''
Ach_eqs_RR = '''
ds_ach/dt = -s_ach/tau_ach : 1 (clock-driven)
Isyn_RR_post = -s_ach*(v-E_ach):1 (summed)
wach : 1 
'''
Ach_eqs_EP = '''
ds_ach/dt = -s_ach/tau_ach : 1 (clock-driven)
Isyn_EP_post = -s_ach*(v-E_ach):1 (summed)
wach : 1 
'''
Ach_eqs_PE = '''
ds_ach/dt = -s_ach/tau_ach : 1 (clock-driven)
Isyn_PE_post = -s_ach*(v-E_ach):1 (summed)
wach : 1 
'''


#set neurongroup and subunit
EPG = NeuronGroup(54,model=eqs,method='euler',threshold='v>Vth', reset='v =Vr',refractory='1*ms')
R = NeuronGroup(3,model=eqs,method='euler',threshold='v>Vth', reset='v =Vr',refractory='1*ms')
PEN = NeuronGroup(48,model=eqs,method='euler',threshold='v>Vth',reset='v=Vr',refractory='1*ms')
EPG_groups = [] # Stores NeuronGroups, one for each population
PEN_groups = []

#Create dictionary of subgroups
for r in range(0, len(subnum_EPG)):
    EPG_groups.append(EPG[nn_cum[r]:nn_cum[r+1]])
EPG_dict = dict(zip(subname_EPG, subnum_EPG))

for r1 in range(0,len(subnum_PEN)):
    PEN_groups.append(PEN[np_cum[r1]:np_cum[r1+1]])
PEN_dict = dict(zip(subname_PEN, subnum_PEN))
# Stores connections
EPG_syngroup = []
for k in range(0,len(subnum_EPG)):
    EPG_syngroup.append(Synapses(EPG_groups[k],EPG_groups[k],Ach_eqs_EE,on_pre = 's_ach += w_EPG', method ='euler'))
    EPG_syngroup[k].connect(condition ='i!=j')

EPG_R_syngroup = Synapses(EPG,R,Ach_eqs_EI,on_pre = 's_ach += w_EI',method ='euler')
for a in range(0,3):
    for b in range(0,54):
        EPG_R_syngroup.connect(i = b , j = a )

R_EPG_syngroup = Synapses(R,EPG,GABA_eqs_IE,on_pre = 's_GABAA += w_IE',method ='euler')
for a1 in range(0,3):
    for b1 in range(0,54):
        R_EPG_syngroup.connect(i = a1 , j = b1 )

R_syngroup = Synapses(R,R,Ach_eqs_RR,on_pre = 's_ach += w_RR',method ='euler') #excitatory???
R_syngroup.connect(condition = 'i!=j')

EPG_PEN_syngroup = []
for a2 in range(0,8):
    EPG_PEN_syngroup.append(Synapses(EPG_groups[a2],PEN_groups[a2],Ach_eqs_EP,on_pre='s_ach += w_EP',method ='euler'))
    EPG_PEN_syngroup[a2].connect(condition = 'i!=j')
for a3 in range(10,18):
    EPG_PEN_syngroup.append(Synapses(EPG_groups[a3],PEN_groups[a3-2],Ach_eqs_EP,on_pre='s_ach += w_EP',method ='euler'))
    EPG_PEN_syngroup[a3-2].connect(condition = 'i!=j')

PEN_EPG_syngroup = []

for b1 in range(0,25,4):
    c1 = int(b1/4)
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1+1],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1+2],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1+9],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1+10],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))


for b1 in range(28,53,4):
    c1 = int(b1/4)+2
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1-8],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1-7],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1+1],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
    PEN_EPG_syngroup.append(Synapses(PEN_groups[c1],EPG_groups[c1+2],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))


PEN_EPG_syngroup.append(Synapses(PEN_groups[7],EPG_groups[0],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[7],EPG_groups[1],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[7],EPG_groups[8],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[7],EPG_groups[9],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[7],EPG_groups[16],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[7],EPG_groups[17],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[8],EPG_groups[0],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[8],EPG_groups[1],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[8],EPG_groups[8],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[8],EPG_groups[9],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[8],EPG_groups[16],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))
PEN_EPG_syngroup.append(Synapses(PEN_groups[8],EPG_groups[17],Ach_eqs_PE,on_pre='s_ach += w_PE', method ='euler'))

for d1 in range(0,len(PEN_EPG_syngroup)):
    PEN_EPG_syngroup[d1].connect(condition = 'i!=j')

S0 = SpikeMonitor(EPG_groups[0])
S1 = SpikeMonitor(EPG_groups[1])
S2 = SpikeMonitor(EPG_groups[2])
S3 = SpikeMonitor(EPG_groups[3])
S4 = SpikeMonitor(EPG_groups[4])
S5 = SpikeMonitor(EPG_groups[5])
S6 = SpikeMonitor(EPG_groups[6])
S7 = SpikeMonitor(EPG_groups[7])
S8 = SpikeMonitor(EPG_groups[8])
S9 = SpikeMonitor(EPG_groups[9])
S10 = SpikeMonitor(EPG_groups[10])
S11 = SpikeMonitor(EPG_groups[11])
S12 = SpikeMonitor(EPG_groups[12])
S13 = SpikeMonitor(EPG_groups[13])
S14 = SpikeMonitor(EPG_groups[14])
S15 = SpikeMonitor(EPG_groups[15])
S16 = SpikeMonitor(EPG_groups[16])
S17 = SpikeMonitor(EPG_groups[17])
store()

In [35]:
restore()
#input

output_rates_0 = []
output_rates_1 = []
output_rates_2 = []
output_rates_3 = []
output_rates_4 = []
output_rates_5 = []


PEN_groups[3].I = 0.05
run(1*second)
output_rates_0.append(S0.num_spikes/3)
output_rates_1.append(S1.num_spikes/3)
output_rates_2.append(S2.num_spikes/3)
output_rates_3.append(S3.num_spikes/3)
output_rates_4.append(S4.num_spikes/3)
output_rates_5.append(S5.num_spikes/3)

PEN_groups[3].I = 0
for time1 in range(0,5):
    restore()
    run(1*second)
    output_rates_0.append(S0.num_spikes)
    output_rates_1.append(S1.num_spikes)
    output_rates_2.append(S2.num_spikes)
    output_rates_3.append(S3.num_spikes)
    output_rates_4.append(S4.num_spikes)
    output_rates_5.append(S5.num_spikes)



print(output_rates_0)
print(output_rates_1)
print(output_rates_2)
print(output_rates_3)
print(output_rates_4)
print(output_rates_5)

[822.6666666666666, 2473, 2465, 2465, 2467, 2466]
[823.6666666666666, 2465, 2465, 2468, 2473, 2468]
[823.0, 2468, 2466, 2468, 2467, 2468]
[822.3333333333334, 2465, 2474, 2472, 2477, 2474]
[824.3333333333334, 2470, 2465, 2467, 2468, 2470]
[822.0, 2464, 2471, 2473, 2475, 2469]
